# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mhamid,29.8200,-5.7200,108.05,9,5,7.34,MA,Sat Aug 19 11:12:09 2023
1,1,san antonio de pale,-1.4014,5.6325,73.40,81,90,10.76,GQ,Sat Aug 19 11:12:10 2023
2,2,kenai,60.5544,-151.2583,49.08,93,0,4.61,US,Sat Aug 19 11:12:11 2023
3,3,kristiansund,63.1115,7.7320,77.65,38,1,9.22,NO,Sat Aug 19 11:12:11 2023
4,4,gloucester point,37.2540,-76.4969,82.29,45,0,6.91,US,Sat Aug 19 11:12:12 2023


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
#%%capture --no-display
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    size = "Humidity",
    color = "City",
    geo = True,
    tiles = "OSM",
    alpha=1,
    frame_width = 700,
    frame_height = 500,
    scale = 1    
)
map_plot_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
#locate rows using .loc that meet criteria for ideal city
ideal_cities = city_data_df.loc[(city_data_df["Max Temp"]>30) & 
                            (city_data_df["Max Temp"]<95) &
                            (city_data_df["Wind Speed"]<4.5) & 
                            (city_data_df["Cloudiness"]==0)]

# Drop any rows with null values
ideal_cities = ideal_cities.dropna(how='any')

# Display sample data
ideal_cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,19,hamilton,39.1834,-84.5333,79.77,57,0,3.44,US,Sat Aug 19 11:12:19 2023
86,86,brookings,44.3114,-96.7984,82.74,73,0,0.00,US,Sat Aug 19 11:10:31 2023
107,107,eureka,40.8021,-124.1637,56.01,76,0,3.69,US,Sat Aug 19 11:08:27 2023
132,132,bandar-e genaveh,29.5791,50.5170,94.71,61,0,4.25,IR,Sat Aug 19 11:13:26 2023
209,209,mo i rana,66.3128,14.1428,75.67,64,0,3.44,NO,Sat Aug 19 11:11:47 2023


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
19,hamilton,US,39.1834,-84.5333,57,
86,brookings,US,44.3114,-96.7984,73,
107,eureka,US,40.8021,-124.1637,76,
132,bandar-e genaveh,IR,29.5791,50.5170,61,
209,mo i rana,NO,66.3128,14.1428,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
#Set the Radius as <10,000 from Lat & Lng

radius = 10000
params = {
    "categories":"accommodation.hotel",
    "api_key":geoapify_key,
    "limit":30
}

# Print a message to follow up the hotel search
print("Starting hotel search")


# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    Lat=row["Lat"]
    Long=row["Lng"]
    
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Long},{Lat},{radius}"
    params["bias"] = f"proximity:{Long},{Lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hamilton - nearest hotel: North Vista Manor
brookings - nearest hotel: Quality Inn
eureka - nearest hotel: Eureka Inn, Trademark Collection by Wyndham
bandar-e genaveh - nearest hotel: منزل دکتور سلمان
mo i rana - nearest hotel: Scandic Meyergården
ocean city - nearest hotel: Watson's Regency Suites
hasaki - nearest hotel: 大新旅館
acapulco de juarez - nearest hotel: Hotel del Valle
smithers - nearest hotel: Sunshine Inn Hotel
bemidji - nearest hotel: Hampton Inn and Suites
mashhad - nearest hotel: هتل تعاون
sinop - nearest hotel: No hotel found
woolgoolga - nearest hotel: Darlington Beach Resort
chernyanka - nearest hotel: Гостиница Центральная
mount isa - nearest hotel: Ibis Styles
tateyama - nearest hotel: No hotel found
narrabri - nearest hotel: Tattersall's Hotel
uychi - nearest hotel: No hotel found
coquitlam - nearest hotel: Poco Inn & Suites Hotel & Conference Centre
alamosa - nearest hotel: 40 Winks Inn
katsuura - nearest hotel: 潮騒の宿
cambria - nearest hotel: 

,City,Country,Lat,Lng,Humidity,Hotel Name
19,hamilton,US,39.1834,-84.5333,57,North Vista Manor
86,brookings,US,44.3114,-96.7984,73,Quality Inn
107,eureka,US,40.8021,-124.1637,76,"Eureka Inn, Trademark Collection by Wyndham"
132,bandar-e genaveh,IR,29.5791,50.5170,61,منزل دکتور سلمان
209,mo i rana,NO,66.3128,14.1428,64,Scandic Meyergården
242,ocean city,US,39.2776,-74.5746,51,Watson's Regency Suites
251,hasaki,JP,35.7333,140.8333,88,大新旅館
260,acapulco de juarez,MX,16.8634,-99.8901,66,Hotel del Valle
300,smithers,CA,54.7804,-127.1743,76,Sunshine Inn Hotel
341,bemidji,US,47.4736,-94.8803,70,Hampton Inn and Suites


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display

# Configure the map plot
map_plot_2 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    size = "Humidity",
    color = "City",
    geo = True,
    tiles = "OSM",
    alpha=1,
    frame_width = 700,
    frame_height = 500,
    scale = 1    
) * \
hotel_df.hvplot.labels(
    x='Lng', 
    y='Lat', 
    text='Hotel Name', 
    text_baseline='bottom', 
    #hover=TRUE
) * \
hotel_df.hvplot.labels(
    x='Lng', 
    y='Lat', 
    text='Country', 
    text_baseline='top', 
    #hover=TRUE
)

# Display the map
map_plot_2


:Overlay
   .Tiles.I   :Tiles   [x,y]
   .Points.I  :Points   [Lng,Lat]   (City,Humidity)
   .Labels.I  :Labels   [Lng,Lat]   (Hotel Name)
   .Labels.II :Labels   [Lng,Lat]   (Country)